In [ ]:
from os import listdir
from os.path import isfile, join
import os
import json
import zipfile
import re
import pkgutil
import os
import encodings

class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

        
# Encodings function
def all_encodings():
    modnames = set(
        [modname for importer, modname, ispkg in pkgutil.walk_packages(
            path=[os.path.dirname(encodings.__file__)], prefix='')])
    aliases = set(encodings.aliases.aliases.values())
    return modnames.union(aliases)

# file_type function requirements

magic_dict = {
    "\x1f\x8b\x08": "gz",
    "\x42\x5a\x68": "bz2",
    "\x50\x4b\x03\x04": "zip"
    }

max_len = max(len(x) for x in magic_dict)

def file_type(filename_in):
    encodings = all_encodings()
    for enc in encodings:
        try:
            with open(filename_in, encoding=enc) as f_in:
                file_start = f_in.read(max_len)
        except:
            pass
    for magic, filetype in magic_dict.items():
        if file_start.startswith(magic):
            return filetype
    return "no match"

#os.walk(directory)
pcaps_dir='/home/cc/Contagio'
directories = [x[0] for x in os.walk(pcaps_dir)]
directories = [ x for x in directories if "MACOSX" not in x ]

#Variables
cnt_pcap=0
cnt_p=0
cnt_wrong_pwd=0
my_dict = AutoVivification()

for i_dir in range(1,len(directories)):
    print(directories[i_dir] + "\n")
    mypath = directories[i_dir]
    
    #onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.endswith('.zip') or f.endswith('.ZIP')]
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and file_type(mypath + "/" + f)=="zip"]
    non_compressed_files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and file_type(mypath + "/" + f)=="no match"]
    #onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for file_x in onlyfiles:
        fullpath = mypath + "/" + file_x
        print(fullpath)
        print(file_type(fullpath))
        print("\n")
    if not os.path.exists(directories[i_dir] + "/unziped"):
        os.makedirs(directories[i_dir] + "/unziped")

    destination_dir = directories[i_dir] + "/unziped"
    for file in onlyfiles:
        print("\n"+ directories[i_dir] + "/" + file)
        path_to_zip_file = directories[i_dir] + "/" + file
        try:
            cnt_pcap=cnt_pcap+1
            zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
            zip_ref.extractall(destination_dir)
            zip_ref.close()
            print("Extraction Succesful")
            my_dict[directories[i_dir]]['Successful'][file]=None
            
        except:
            cnt_pcap=cnt_pcap-1
            print("Password Required")
            cnt_p=cnt_p+1
            try:
                zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
                zip_ref.extractall(destination_dir,None,pwd=b'infected666p')
                zip_ref.close()
                cnt_p=cnt_p-1
                cnt_pcap=cnt_pcap+1
                print("Extraction Succesful")
                my_dict[directories[i_dir]]['Successful'][file]=None
            except:
                cnt_wrong_pwd=cnt_wrong_pwd+1
                print("Invalid Password")
                my_dict[directories[i_dir]]['Pending'][file]=None
            
    onlyfiles=[]
    

print("\nFinal Results")
print("Successful Extractions: %s" %cnt_pcap)
print("Unsuccessful Extractions: %s" %cnt_p)
print("Invalid Passwords: %s" %cnt_wrong_pwd)
print("\nDictionary")
print(json.dumps(my_dict, sort_keys=True, indent=4))
print("\nCompressed files")
print(onlyfiles)
print("\nNon_compressed_files")
print(non_compressed_files)